#### complaint agent

this notebook creates a complaint handling agent with tools to investigate orders and make decisions about credits, investigations, or escalations

#### Tool & View Registration

In [ ]:
%sql
CREATE SCHEMA IF NOT EXISTS ${CATALOG}.ai;

In [ ]:
%sql
CREATE OR REPLACE VIEW ${CATALOG}.ai.order_delivery_times_per_location_view AS
WITH order_times AS (
  SELECT
    order_id,
    location,
    MAX(CASE WHEN event_type = 'order_created' THEN try_to_timestamp(ts) END) AS order_created_time,
    MAX(CASE WHEN event_type = 'delivered' THEN try_to_timestamp(ts) END) AS delivered_time
  FROM
    ${CATALOG}.lakeflow.all_events
  WHERE
    try_to_timestamp(ts) >= CURRENT_TIMESTAMP() - INTERVAL 1 DAY
  GROUP BY
    order_id,
    location
),
total_order_times AS (
  SELECT
    order_id,
    location,
    (UNIX_TIMESTAMP(delivered_time) - UNIX_TIMESTAMP(order_created_time)) / 60 AS total_order_time_minutes
  FROM
    order_times
  WHERE
    order_created_time IS NOT NULL
    AND delivered_time IS NOT NULL
)
SELECT
  location,
  PERCENTILE(total_order_time_minutes, 0.50) AS P50,
  PERCENTILE(total_order_time_minutes, 0.75) AS P75,
  PERCENTILE(total_order_time_minutes, 0.99) AS P99
FROM
  total_order_times
GROUP BY
  location

In [ ]:
%sql
CREATE OR REPLACE FUNCTION ${CATALOG}.ai.get_order_overview(oid STRING COMMENT 'The unique order identifier to retrieve information for')
RETURNS TABLE (
  order_id STRING COMMENT 'The order id',
  location STRING COMMENT 'Order location',
  items_json STRING COMMENT 'JSON array of ordered items with details',
  customer_address STRING COMMENT 'Customer delivery address',
  brand_id BIGINT COMMENT 'Brand ID for the order',
  order_created_ts TIMESTAMP COMMENT 'When the order was created'
)
COMMENT 'Returns basic order information including items, location, and customer details'
RETURN
  WITH order_created_events AS (
    SELECT
      order_id,
      location,
      get_json_object(body, '$.items') as items_json,
      get_json_object(body, '$.customer_addr') as customer_address,
      -- Extract brand_id from first item in the order
      CAST(get_json_object(get_json_object(body, '$.items[0]'), '$.brand_id') AS BIGINT) as brand_id,
      try_to_timestamp(ts) as order_created_ts
    FROM ${CATALOG}.lakeflow.all_events
    WHERE order_id = oid AND event_type = 'order_created'
    LIMIT 1
  )
  SELECT
    order_id,
    location,
    items_json,
    customer_address,
    brand_id,
    order_created_ts
  FROM order_created_events;

In [ ]:
%sql
CREATE OR REPLACE FUNCTION ${CATALOG}.ai.get_order_timing(oid STRING COMMENT 'The unique order identifier to get timing information for')
RETURNS TABLE (
  order_id STRING COMMENT 'The order id',
  order_created_ts TIMESTAMP COMMENT 'When the order was created',
  delivered_ts TIMESTAMP COMMENT 'When the order was delivered (NULL if not delivered)',
  delivery_duration_minutes FLOAT COMMENT 'Time from order creation to delivery in minutes (NULL if not delivered)',
  delivery_status STRING COMMENT 'Current delivery status: delivered, in_progress, or unknown'
)
COMMENT 'Returns timing information for a specific order'
RETURN
  WITH order_events AS (
    SELECT
      order_id,
      event_type,
      try_to_timestamp(ts) as event_ts
    FROM ${CATALOG}.lakeflow.all_events
    WHERE order_id = oid
  ),
  timing_summary AS (
    SELECT
      order_id,
      MIN(CASE WHEN event_type = 'order_created' THEN event_ts END) as order_created_ts,
      MAX(CASE WHEN event_type = 'delivered' THEN event_ts END) as delivered_ts
    FROM order_events
    GROUP BY order_id
  )
  SELECT
    order_id,
    order_created_ts,
    delivered_ts,
    CASE
      WHEN delivered_ts IS NOT NULL AND order_created_ts IS NOT NULL THEN
        CAST((UNIX_TIMESTAMP(delivered_ts) - UNIX_TIMESTAMP(order_created_ts)) / 60 AS FLOAT)
      ELSE NULL
    END as delivery_duration_minutes,
    CASE
      WHEN delivered_ts IS NOT NULL THEN 'delivered'
      WHEN order_created_ts IS NOT NULL THEN 'in_progress'
      ELSE 'unknown'
    END as delivery_status
  FROM timing_summary;

In [ ]:
%sql
CREATE OR REPLACE FUNCTION ${CATALOG}.ai.get_location_timings(loc STRING COMMENT 'Location name as a string')
RETURNS TABLE (
  location STRING COMMENT 'Location of the order source',
  P50 FLOAT COMMENT '50th percentile delivery time in minutes',
  P75 FLOAT COMMENT '75th percentile delivery time in minutes',
  P99 FLOAT COMMENT '99th percentile delivery time in minutes'
)
COMMENT 'Returns the 50/75/99th percentile of delivery times for a location to benchmark order timing'
RETURN
  SELECT location, P50, P75, P99
  FROM ${CATALOG}.ai.order_delivery_times_per_location_view AS odlt
  WHERE odlt.location = loc;

#### Model

In [ ]:
%pip install -U -qqqq mlflow-skinny[databricks] langgraph==0.3.4 databricks-langchain databricks-agents uv
dbutils.library.restartPython()

In [ ]:
CATALOG = dbutils.widgets.get("CATALOG")
LLM_MODEL = dbutils.widgets.get("LLM_MODEL")

In [ ]:
import re
from IPython.core.magic import register_cell_magic

@register_cell_magic
def writefilev(line, cell):
    """
    %%writefilev file.py
    Allows {{var}} substitutions while leaving normal {} intact.
    """
    filename = line.strip()

    def replacer(match):
        expr = match.group(1)
        return str(eval(expr, globals(), locals()))

    # Replace only double braces {{var}}
    content = re.sub(r"\{\{(.*?)\}\}", replacer, cell)

    with open(filename, "w") as f:
        f.write(content)
    print(f"Wrote file with substitutions: {filename}")

In [ ]:
%%writefilev agent.py
from typing import Any, Generator, Optional, Sequence, Union

import mlflow
from databricks_langchain import (
    ChatDatabricks,
    VectorSearchRetrieverTool,
    DatabricksFunctionClient,
    UCFunctionToolkit,
    set_uc_function_client,
)
from langchain_core.language_models import LanguageModelLike
from langchain_core.runnables import RunnableConfig, RunnableLambda
from langchain_core.tools import BaseTool
from langgraph.graph import END, StateGraph
from langgraph.graph.graph import CompiledGraph
from langgraph.graph.state import CompiledStateGraph
from langgraph.prebuilt.tool_node import ToolNode
from mlflow.langchain.chat_agent_langgraph import ChatAgentState, ChatAgentToolNode
from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import (
    ChatAgentChunk,
    ChatAgentMessage,
    ChatAgentResponse,
    ChatContext,
)

mlflow.langchain.autolog()

client = DatabricksFunctionClient()
set_uc_function_client(client)

############################################
# Define your LLM endpoint and system prompt
############################################
LLM_ENDPOINT_NAME = f"{{LLM_MODEL}}"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME)

system_prompt = """You are ComplaintAgent, a unified customer service agent for Chef Casper's multi-brand ghost kitchen operation.

You handle customer complaints by investigating orders and making data-driven decisions about credits, investigations, or escalations.

Process:
1. Extract order_id from the customer complaint
2. Call `get_order_overview(order_id)` to get basic order details
3. Call `get_order_timing(order_id)` to get delivery timeline
4. If timing-related complaint, call `get_location_timings(location)` for context
5. Classify the complaint and make a decision

Decision Framework:
- AUTO-CREDIT: Clear, minor issues (late delivery >P75, missing low-value items)
- INVESTIGATE: Uncertain or moderate issues (food quality claims, service complaints, billing)
- ESCALATE: Severe issues (safety concerns, threats, high-value claims)

Be helpful but data-driven. Only offer credits when justified by evidence.

Always provide:
- order_id (the order identifier from the complaint)
- complaint_category (delivery_delay, missing_items, food_quality, service_issue, billing, other)
- decision (auto_credit, investigate, escalate)
- credit_amount (if applicable)
- rationale (clear explanation based on order data)
- customer_response (professional response text)

Output a single-line JSON with these fields. No extra text or markdown."""

###############################################################################
## Define tools for your agent
###############################################################################
tools = []

uc_tool_names = [f"{{CATALOG}}.ai.get_order_overview", 
                 f"{{CATALOG}}.ai.get_order_timing",
                 f"{{CATALOG}}.ai.get_location_timings"]
uc_toolkit = UCFunctionToolkit(function_names=uc_tool_names)
tools.extend(uc_toolkit.tools)

#####################
## Define agent logic
#####################


def create_tool_calling_agent(
    model: LanguageModelLike,
    tools: Union[Sequence[BaseTool], ToolNode],
    system_prompt: Optional[str] = None,
) -> CompiledGraph:
    model = model.bind_tools(tools)

    # Define the function that determines which node to go to
    def should_continue(state: ChatAgentState):
        messages = state["messages"]
        last_message = messages[-1]
        # If there are function calls, continue. else, end
        if last_message.get("tool_calls"):
            return "continue"
        else:
            return "end"

    if system_prompt:
        preprocessor = RunnableLambda(
            lambda state: [{"role": "system", "content": system_prompt}]
            + state["messages"]
        )
    else:
        preprocessor = RunnableLambda(lambda state: state["messages"])
    model_runnable = preprocessor | model

    def call_model(
        state: ChatAgentState,
        config: RunnableConfig,
    ):
        response = model_runnable.invoke(state, config)

        return {"messages": [response]}

    workflow = StateGraph(ChatAgentState)

    workflow.add_node("agent", RunnableLambda(call_model))
    workflow.add_node("tools", ChatAgentToolNode(tools))

    workflow.set_entry_point("agent")
    workflow.add_conditional_edges(
        "agent",
        should_continue,
        {
            "continue": "tools",
            "end": END,
        },
    )
    workflow.add_edge("tools", "agent")

    return workflow.compile()


class LangGraphChatAgent(ChatAgent):
    def __init__(self, agent: CompiledStateGraph):
        self.agent = agent

    def predict(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> ChatAgentResponse:
        request = {"messages": self._convert_messages_to_dict(messages)}

        messages = []
        for event in self.agent.stream(request, stream_mode="updates"):
            for node_data in event.values():
                messages.extend(
                    ChatAgentMessage(**msg) for msg in node_data.get("messages", [])
                )
        return ChatAgentResponse(messages=messages)

    def predict_stream(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> Generator[ChatAgentChunk, None, None]:
        request = {"messages": self._convert_messages_to_dict(messages)}
        for event in self.agent.stream(request, stream_mode="updates"):
            for node_data in event.values():
                yield from (
                    ChatAgentChunk(**{"delta": msg}) for msg in node_data["messages"]
                )


# Create the agent object, and specify it as the agent object to use when
# loading the agent back for inference via mlflow.models.set_model()
agent = create_tool_calling_agent(llm, tools, system_prompt)
AGENT = LangGraphChatAgent(agent)
mlflow.models.set_model(AGENT)

In [ ]:
# get an actual order_id for input example
sample_order_id = spark.sql(f"""
    SELECT order_id 
    FROM {CATALOG}.lakeflow.all_events 
    WHERE event_type='delivered'
    LIMIT 1
""").collect()[0]['order_id']

In [ ]:
assert sample_order_id is not None
print(sample_order_id)

In [ ]:
import mlflow
from agent import LLM_ENDPOINT_NAME, tools
from databricks_langchain import VectorSearchRetrieverTool
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint
from pkg_resources import get_distribution
from unitycatalog.ai.langchain.toolkit import UnityCatalogTool

resources = [DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME)]
for tool in tools:
    resources.append(DatabricksFunction(function_name=tool.uc_function_name))

input_example = {
    "messages": [
        {
            "role": "user",
            "content": f"My order was really late! Order ID: {sample_order_id}"
        }
    ]
}

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        name="complaint_agent",
        python_model="agent.py",
        input_example=input_example,
        resources=resources,
        pip_requirements=[
            f"databricks-connect=={get_distribution('databricks-connect').version}",
            f"mlflow=={get_distribution('mlflow').version}",
            f"databricks-langchain=={get_distribution('databricks-langchain').version}",
            f"langgraph=={get_distribution('langgraph').version}",
        ],
    )

mlflow.set_active_model(model_id = logged_agent_info.model_id)

#### eval

In [ ]:
# sample 10 order_ids
complaint_queries = [
    row['order_id'] for row in spark.sql(f"""
        SELECT order_id 
        FROM {CATALOG}.lakeflow.all_events 
        WHERE event_type='delivered'
        LIMIT 10
    """).collect()
]

# wrap in correct input schema with complaint text
data = []
for query in complaint_queries:
    data.append(
        {
            "inputs": {
                "messages": [
                    {
                        "role": "user",
                        "content": f"My order was late and the food quality was poor! Order: {query}",
                    }
                ]
            },
        }
    )

print(data)

In [ ]:
# create guideline, run evals

from mlflow.genai.scorers import Guidelines
import mlflow
import sys
import os
sys.path.append(os.getcwd())

notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
project_directory = os.path.dirname(notebook_path)

# Add the project directory to the system path
sys.path.append(project_directory)

from agent import AGENT

decision_quality = Guidelines(
    name="decision_quality",
    guidelines=[
        "Food quality complaints should be classified as 'investigate', not 'auto_credit'",
        "Missing item complaints should be classified as 'investigate', not 'auto_credit'",
        "Service complaints should be classified as 'investigate', not 'auto_credit'",
        "Legal threats or serious health concerns should be classified as 'escalate'"
    ]
)

results = mlflow.genai.evaluate(
    data=data,
    scorers=[decision_quality],
    predict_fn = lambda messages: AGENT.predict({"messages": messages})
)

#### log complaint agent to `UC`

In [ ]:
mlflow.set_registry_uri("databricks-uc")

UC_MODEL_NAME = f"{CATALOG}.ai.complaint_agent"

# register the model to UC
uc_registered_model_info = mlflow.register_model(
    model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME
)

#### deploy the agent to model serving

In [ ]:
from databricks import agents
deployment_info = agents.deploy(
    model_name=UC_MODEL_NAME, 
    model_version=uc_registered_model_info.version, 
    scale_to_zero=False,
    endpoint_name=f"{dbutils.widgets.get('COMPLAINT_AGENT_ENDPOINT_NAME')}")

In [ ]:
print(deployment_info)

##### record model in state

In [ ]:
# Also add to UC-state
import sys
sys.path.append('../utils')
from uc_state import add

add(dbutils.widgets.get("CATALOG"), "endpoints", deployment_info)